In [1]:
from qwen_agent.tools import TOOL_REGISTRY, MCPManager, BaseTool
import json
from ast import literal_eval


def parse_mcp_tools_config(file_path):
    try:
        with open(file_path, 'r') as f:
            content = f.read()
        # 使用 literal_eval 安全地解析 Python 字面量
        data = literal_eval(content)
        return data
    except Exception as e:
        print(f"解析错误: {e}")
        return None

tool = parse_mcp_tools_config("/home/ma-user/work/RL-Factory/envs/configs/bfcl_mcp_tools.pydata")[0]
# tool = parse_mcp_tools_config("/hpc/home/minruixu/RL-Factory/envs/configs/mcp_tools.pydata")[0]

tools = MCPManager().initConfig(tool)

2025-09-08 19:29:13,025 - mcp_manager.py - 141 - INFO - Initializing MCP tools from mcp servers: ['file_system', 'search']
2025-09-08 19:29:13,031 - mcp_manager.py - 370 - INFO - Initializing a MCP stdio_client, if this takes forever, please check the config of this mcp server: file_system
2025-09-08 19:29:19,016 - mcp_manager.py - 370 - INFO - Initializing a MCP stdio_client, if this takes forever, please check the config of this mcp server: search


In [2]:
# 读取parquet文件
import pyarrow.parquet as pq

parquet_file = pq.ParquetFile('/home/ma-user/work/RL-Factory/data/BFCL/multi-turn/train.parquet')
data = parquet_file.read().to_pandas()

In [3]:
data.head()

,id,question,golden_answers,data_source,prompt,ability,reward_model,extra_info
0,multi_turn_base_0_turn_0_0,Move 'final_report.pdf' within document direct...,"[""file_system-cd(folder='document')"", ""file_sy...",BFCL_multi_turn_base,"[{""content"": ""You are a helpful assistant that...",tool_use,"{""ground_truth"": ""[\""file_system-cd(folder='do...","{""index"": 0, ""split"": ""train"", ""involved_class..."
1,multi_turn_base_0_turn_1_0,Perform a detailed search using grep to identi...,"[""file_system-cd(folder='temp')"", ""file_system...",BFCL_multi_turn_base,"[{""content"": ""You are a helpful assistant that...",tool_use,"{""ground_truth"": ""[\""file_system-cd(folder='te...","{""index"": 1, ""split"": ""train"", ""involved_class..."
2,multi_turn_base_0_turn_2_0,Upon identifying the requisite 'budget analysi...,"[""file_system-sort(file_name='final_report.pdf...",BFCL_multi_turn_base,"[{""content"": ""You are a helpful assistant that...",tool_use,"{""ground_truth"": ""[\""file_system-sort(file_nam...","{""index"": 2, ""split"": ""train"", ""involved_class..."
3,multi_turn_base_0_turn_3_0,Move 'previous_report.pdf' in document directo...,"[""file_system-cd(folder='..')"", ""file_system-m...",BFCL_multi_turn_base,"[{""content"": ""You are a helpful assistant that...",tool_use,"{""ground_truth"": ""[\""file_system-cd(folder='.....","{""index"": 3, ""split"": ""train"", ""involved_class..."
4,multi_turn_base_1_turn_0_0,I am alex. Check if the current directory is u...,"[""file_system-ls(show_hidden=True)""]",BFCL_multi_turn_base,"[{""content"": ""You are a helpful assistant that...",tool_use,"{""ground_truth"": ""[\""file_system-ls(show_hidde...","{""index"": 4, ""split"": ""train"", ""involved_class..."


In [4]:

def get_initial_config(data, row_index=0):
    """
    从parquet数据中读取指定行的initial_config
    
    Args:
        data: pandas DataFrame
        row_index: 行索引
    
    Returns:
        dict: initial_config字典
    """
    try:
        extra_info_str = data['extra_info'][row_index]
        extra_info = json.loads(extra_info_str)
        return extra_info.get('initial_config', {})
    except Exception as e:
        print(f"读取第{row_index}行initial_config时出错: {e}")
        return {}

# 测试函数
config = get_initial_config(data, 0)
print("使用函数读取的initial_config:")
print(json.dumps(config, indent=2, ensure_ascii=False))

使用函数读取的initial_config:
{
  "posting": {
    "username": "analyst_pro",
    "password": "Kj8#mP9$vL2",
    "authenticated": false,
    "tweets": {
      "0": {
        "id": 0,
        "username": "analyst_pro",
        "content": "Just finished analyzing the reports!",
        "tags": [
          "#analysis",
          "#reports"
        ],
        "mentions": []
      },
      "1": {
        "id": 1,
        "username": "analyst_pro",
        "content": "Budget analysis insights coming soon!",
        "tags": [
          "#budget",
          "#analysis",
          "#insights"
        ],
        "mentions": []
      },
      "2": {
        "id": 2,
        "username": "analyst_pro",
        "content": "Stay tuned for more updates!",
        "tags": [
          "#updates",
          "#staytuned"
        ],
        "mentions": []
      }
    },
    "comments": {},
    "retweets": {},
    "following_list": [
      "alice",
      "bob"
    ],
    "tweet_counter": 3
  },
  "file_system": {


In [6]:
file_system = config['file_system']
print(file_system)

{'root': {'workspace': {'type': 'directory', 'contents': {'document': {'type': 'directory', 'contents': {'final_report.pdf': {'type': 'file', 'content': 'Year2024 This is the final report content including budget analysis and other sections.'}, 'previous_report.pdf': {'type': 'file', 'content': 'Year203 This is the previous report content with different budget analysis.'}}}, 'archive': {'type': 'directory', 'contents': {}}}}}}


In [7]:
tools[0].call(json.dumps({"scenario": file_system}))

'Sucessfully load from scenario.'